In [1]:
import json
import lasagne
from lasagne import nonlinearities
import numpy as np
import theano

In [2]:
dataT = open("allenNetwork.json")
s = dataT.read()
obj = json.loads(s)

In [3]:
listOfLayers = obj['layers']

In [4]:
# build input layer
inputLayer = listOfLayers[0]
input_var = theano.tensor.tensor4('inputs')
l_in = lasagne.layers.InputLayer(shape=(None, 8, 25, 25), input_var=input_var)

In [6]:
layerWeightMatrices = []
convolutionLayers = [l_in]

for layer in listOfLayers:
    if(layer['layer_type'] == 'conv'):
        filterWeights = layer['filters']
        
        noOfFilters = len(filterWeights) 
        sx = filterWeights[0]['sx']
        sy = filterWeights[0]['sy']
        depth = filterWeights[0]['depth']
        print depth, sx, sy, layer['stride'], layer['pad']
        weightArray = np.zeros((noOfFilters,depth,sx,sy))
        
        for index in range(0, len(filterWeights)):
            weights = np.array(filterWeights[index]['w'])
            weights = weights.reshape((depth,sx,sy))
            weightArray[index] = weights
        
        biases = np.array(layer['biases']['w'])
        
        convLayer = lasagne.layers.Conv2DLayer(incoming = convolutionLayers[-1], num_filters = noOfFilters, filter_size = (sx,sy), stride = layer['stride'], pad = layer['pad'], W = weightArray, b = biases, nonlinearity = lasagne.nonlinearities.rectify)
        convolutionLayers.append(convLayer)
        layerWeightMatrices.append(weightArray)

8 7 7 1 0
64 5 5 1 2
64 5 5 1 2
64 5 5 1 2
48 5 5 1 2
48 5 5 1 2
32 5 5 1 2


In [7]:
for layer in listOfLayers:
    if(layer['layer_type'] == 'fc'):
        num_units  = len(layer['filters'])
        num_inputs = layer['num_inputs']
        weights = np.zeros((num_units, num_inputs))
        for i in range(0, num_units):
            weights[i] = np.array(layer['filters'][i]['w'])
        weights = np.transpose(weights)
        biases = np.array(layer['biases']['w'])
        fcLayer = lasagne.layers.DenseLayer(incoming = convolutionLayers[-1], num_units = num_units, W = weights, b = biases, nonlinearity=lasagne.nonlinearities.softmax)

In [8]:
prediction_ = lasagne.layers.get_output(fcLayer, deterministic=True)
predict_function = theano.function([input_var], prediction_)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '10520' (I am process '9140')
INFO (theano.gof.compilelock): Refreshing lock C:\Users\haresh\AppData\Local\Theano\compiledir_Windows-8-6.2.9200-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.10-64\lock_dir\lock
INFO:theano.gof.compilelock:Refreshing lock C:\Users\haresh\AppData\Local\Theano\compiledir_Windows-8-6.2.9200-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.10-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\haresh\AppData\Local\Theano\compiledir_Windows-8-6.2.9200-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.10-64\lock_dir\lock
INFO:theano.gof.compilelock:Refreshing lock C:\Users\haresh\AppData\Local\Theano\compiledir_Windows-8-6.2.9200-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.10-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\haresh\AppData\Local\Theano\compiledir_Windows-8-6.2.9200-Intel64_Family_6_Model_58_Stepping_

In [9]:
#Test with an input
inputBoard = np.zeros((8,25,25))
# Set all squares in the edge channel to 1
for x in range(0,25):
    for y in range(0,25):
        if (x < 3 or x >= 22 or y < 3 or y >= 22):
            inputBoard[7][x][y] = 1

In [10]:
inputBoard = inputBoard.reshape(1,8,25,25)
Y_pred = predict_function(inputBoard)
print Y_pred

[[  1.95231664e-03   5.07942633e-03   6.72265353e-03   3.59816450e-03
    5.01914296e-03   7.34517351e-03   3.45435575e-03   7.70903061e-03
    3.39079471e-03   4.76035503e-03   6.73691324e-03   5.36233268e-03
    3.31208880e-03   5.50938950e-03   7.09632841e-03   4.79576522e-03
    1.79210897e-03   4.40349411e-03   2.13841782e-03   3.69472251e-03
    2.52268933e-03   3.29701430e-03   2.03236106e-03   2.87592417e-03
    2.63902938e-03   2.86437668e-03   2.01728146e-03   2.13766565e-03
    1.07577354e-03   1.51985198e-03   2.28582073e-03   7.27347049e-03
    2.41460069e-03   2.73473463e-03   1.56626751e-03   5.33237049e-03
    1.40895882e-03   3.50426548e-03   6.04953981e-03   2.53632338e-03
    4.34554518e-03   5.15705230e-03   4.71974242e-03   2.65066457e-03
    2.62768620e-03   2.39993952e-03   4.06131482e-03   2.09249502e-03
    1.65931129e-03   2.12184176e-03   2.48591461e-03   2.42276468e-03
    2.63992725e-03   3.15786901e-03   4.44094341e-03   3.72923891e-03
    6.45179851e-03  

In [13]:
print np.argmax(Y_pred[0])

281


In [16]:
print (281/19), (281 % 19)

14 15


In [23]:
boardProbs = Y_pred[0].reshape((19,19))
print np.argmax(boardProbs)
print np.amax(boardProbs)

281
0.0289558169147


In [25]:
print boardProbs[14][15]

0.0289558169147
